In [3600]:
from imutils import contours
import numpy as np
import argparse
import cv2
import pytesseract
import import_ipynb
import myutils
import easyocr

In [3601]:
def cv_show(name,img):
    cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows();

In [3602]:
img = cv2.imread('plate_12.jpeg')

In [3603]:
ratio = img.shape[0]/500.0
orig = img.copy();

img = myutils.resize(orig,height=500)
orig = myutils.resize(orig,height=500)

In [3604]:
cv_show('img',img)
img.shape

(500, 666, 3)

In [3605]:
img=cv2.GaussianBlur(img,(5,5),0)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 10))
blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKern)
cv_show("Tophat", blackhat)

In [3606]:
squareKern = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
light = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, squareKern)
light = cv2.threshold(light, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
cv_show("Light Regions", light)

In [3607]:
gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F,dx=1, dy=0, ksize=-1)
gradX = np.absolute(gradX)
(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = 255 * ((gradX - minVal) / (maxVal - minVal))
gradX = gradX.astype("uint8")
cv_show("Scharr", gradX)


In [3608]:
gradX = cv2.GaussianBlur(gradX, (5, 5), 0)
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKern)
thresh = cv2.threshold(gradX, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
cv_show("Grad Thresh", thresh)

In [3609]:
thresh = cv2.erode(thresh, None, iterations=2)
thresh = cv2.dilate(thresh, None, iterations=2)

In [3610]:
thresh = cv2.bitwise_and(thresh, thresh, mask=light)
thresh = cv2.dilate(thresh, None, iterations=2)
thresh = cv2.erode(thresh, None, iterations=1)
cv_show("Final", thresh)

In [3611]:
cnts,hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
draw_img = img.copy();
cv2.drawContours(draw_img,cnts,-1,(0,0,255),3)
cv_show('cnts',draw_img)

In [3612]:
lpCnt = None
roi = None
cnts = sorted(cnts,key=cv2.contourArea,reverse=True);
		# loop over the license plate candidate contours
for c in cnts:
    # compute the bounding box of the contour and then use
    # the bounding box to derive the aspect ratio
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    print(ar)
    roi = img[y:y+h,x:x+w]
    cv_show("roi is ",roi)
    if ar >= 3 and ar <= 6:
        # store the license plate contour and extract the
        # license plate from the grayscale image and then
        # threshold it
        lpCnt = c
        licensePlate = img[y:y + h, x:x+ w]
        cv_show('licenseplate',licensePlate)
        break

2.23943661971831
4.333333333333333


In [3613]:
tmp = licensePlate.copy()

In [3614]:
dilate_img = cv2.dilate(licensePlate,np.ones((7,7),np.uint8))
bg_img = cv2.medianBlur(dilate_img,21)
diff_img = 255-cv2.absdiff(tmp,bg_img)
norm_img = diff_img.copy() # Needed for 3.x compatibility
cv2.normalize(diff_img, norm_img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
_, thr_img = cv2.threshold(norm_img, 230, 0, cv2.THRESH_TRUNC)
cv2.normalize(thr_img, thr_img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
licensePlate = thr_img.copy()


In [3615]:
licensePlate = myutils.resize(licensePlate,width=300)

In [3616]:
licensePlate = myutils.resize(licensePlate,height=60)
colored = licensePlate.copy();

In [3617]:
licensePlate = cv2.GaussianBlur(licensePlate,(3,3),0)
licensePlate = cv2.cvtColor(licensePlate,cv2.COLOR_BGR2GRAY)
licensePlate = cv2.threshold(licensePlate, 0, 255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

In [3618]:
# squareKern = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
lp_close = cv2.morphologyEx(licensePlate,cv2.MORPH_CLOSE,squareKern)
lp_close = cv2.morphologyEx(lp_close,cv2.MORPH_OPEN,squareKern)
lp_close = cv2.erode(lp_close,squareKern,iterations=1)

In [3619]:
adjust = lp_close.copy()
cv_show('lp_close',adjust)

In [3620]:
result = np.zeros(licensePlate.shape, dtype=np.uint8)
cv_show('res',result)

In [3621]:
lp_close = licensePlate.copy()
cv_show('lpclose',lp_close)
contours,hierarchy = cv2.findContours(lp_close,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE);
num_labels,labels ,stats,centroid= cv2.connectedComponentsWithStats(lp_close,8,cv2.CV_32S)


In [3622]:
cv_show('colored',lp_close)

In [3623]:
print(num_labels)
for i in range(1,num_labels):
    x = stats[i,cv2.CC_STAT_LEFT]
    y = stats[i, cv2.CC_STAT_TOP]
    w = stats[i, cv2.CC_STAT_WIDTH]
    h = stats[i, cv2.CC_STAT_HEIGHT]
    area = stats[i, cv2.CC_STAT_AREA]
    (cX,cY) = centroid[i];
    output = colored.copy()
    cv2.rectangle(output,(x,y),(x+w,y+h),(0,255,0),3)
    componentMask = (labels==i).astype("uint8")*255
    ratio = w/float(h);
    
    print("width is ",w)
    print("height is ",h)
    print("area is ",area)
    #26 40 613
    #26,40,556
    #26 40 664
    #24,40,524
    #24,40,461
    #23,40,450
#     cv_show('output',output)
    if ratio >= 0.1 and ratio <= 2.0 and area >=200:
        cv_show('output',output)
        result[y:y+h,x:x+w] = lp_close[y:y+h,x:x+w]

13
width is  193
height is  4
area is  605
width is  24
height is  3
area is  43
width is  24
height is  4
area is  92
width is  260
height is  60
area is  1023
width is  24
height is  38
area is  470
width is  23
height is  37
area is  475
width is  23
height is  38
area is  512
width is  23
height is  37
area is  417
width is  22
height is  36
area is  398
width is  24
height is  36
area is  404
width is  25
height is  37
area is  522
width is  7
height is  5
area is  33


In [3624]:
invert = 255-result
cv_show('invert',invert);

In [3625]:
val = pytesseract.image_to_string(invert, config = '--psm 7')
print(val)

ZG 593 ZH



In [3626]:
x,y,w,h = cv2.boundingRect(lpCnt);
cv2.rectangle(orig,(x,y),(x+w,y+h),(0,0,255),2)
cv2.putText(orig, val, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
cv_show('orig',orig)